Sacamos la esperanza de vida a los 65 por municipio como indicador de longevidad. Luego analizaremos si la diferencia en la esperanza de vida entre municipios correlacionado con algún factor.

*Fuente*: https://www.ine.es/dynt3/inebase/index.htm?padre=1153&capsel=1153

In [2]:
import pandas as pd
file = 'esperanza_vida_65a.csv'

Sacamos el codine:

In [5]:
edv_65 = pd.read_csv(f'input/esperanza_vida_65a.csv',sep=";", decimal=",")
edv_65['codine'] = edv_65.Municipios.str.split(' ',expand=True)[0]
edv_65['municipio'] = edv_65.Municipios.str.split(' ',expand=True)[1]

Simplificamos el literal del sexo:

In [6]:
sexo = {'Ambos sexos':'tot', 'Hombres':'H', 'Mujeres':'M'}
edv_65.Sexo = edv_65.Sexo.map(sexo)

In [7]:
pv_edv_65 = edv_65.pivot(index='codine', columns=['Periodo','Sexo'], values='Total')
pv_edv_65.columns = [c[1]+'_'+str(c[0]) for c in pv_edv_65.columns]
pv_edv_65.head(2)

,tot_2021,tot_2020,tot_2019,tot_2018,tot_2017,tot_2016,tot_2015,tot_2014,H_2021,H_2020,...,H_2015,H_2014,M_2021,M_2020,M_2019,M_2018,M_2017,M_2016,M_2015,M_2014
codine,,,,,,,,,,,,,,,,,,,,,
01059,21.91,22.07,22.09,22.13,21.95,21.68,21.83,21.70,19.55,19.77,...,19.74,19.60,24.10,24.20,24.11,24.10,23.98,23.68,23.71,23.59
02003,20.43,20.40,20.32,21.14,21.00,20.91,20.72,20.72,18.25,18.24,...,18.66,18.79,22.38,22.33,22.21,22.92,22.77,22.70,22.51,22.37


In [9]:
pv_edv_65.to_csv('output/pv_edv_65.csv',index=True)